## Import Libs

In [1]:
# import os
import pandas as pd
import numpy as np
import datetime as dt
import logging
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

from torch import optim
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

In [2]:
from entities import *

### Setting 

In [6]:
logging.basicConfig(level=logging.DEBUG)
logging.getLogger('matplotlib').setLevel(logging.WARNING)

In [7]:
symbols = ['AAPL', 'TSLA', 'MSFT']
basket = Basket(symbols=symbols)
basket.load_all_assets(freq="1d")

DEBUG:entities.basket:Initialized Asset Basket: ['AAPL', 'TSLA', 'MSFT'] with 0 assets which loaded.
INFO:entities.basket:Starting batch load for 3 symbols...
DEBUG:entities.basket:Attempting to load AAPL...
DEBUG:entities.asset:Initialized Asset: AAPL with 2724 rows.
INFO:entities.basket:Successfully loaded AAPL (2724 rows).
DEBUG:entities.basket:Attempting to load TSLA...
DEBUG:entities.asset:Initialized Asset: TSLA with 2760 rows.
INFO:entities.basket:Successfully loaded TSLA (2760 rows).
DEBUG:entities.basket:Attempting to load MSFT...
DEBUG:entities.asset:Initialized Asset: MSFT with 2724 rows.
INFO:entities.basket:Successfully loaded MSFT (2724 rows).
INFO:entities.basket:Batch load complete. Success: 3/3. Total assets in basket: 3


In [ ]:
target_features = ["Close"]
for symbol, asset in basket.assets.items():
    asset.to_returns(log=True, target_features=target_features)